In [ ]:
pip install datasets

In [ ]:
pip install evaluate

In [ ]:
pip install jiwer

In [ ]:
pip install noisereduce

In [ ]:
pip install peft

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from transformers import WhisperForConditionalGeneration, WhisperProcessor
from datasets import load_dataset
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import EncoderDecoderCache
from peft import get_peft_model, LoraConfig
import torch
import librosa
from datasets import Dataset
import re
import noisereduce as nr

**Calling Whisper Open ai**

In [ ]:
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
processor = WhisperProcessor.from_pretrained("openai/whisper-small")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


**Loading the dataset Local**

In [ ]:
# Sample dataset (5 small audio files and transcriptions)
# Path to audio files on a Windows system
"""data1 = [
    {"audio": r"audio_1.wav", "text": r"text_1.txt"},
    {"audio": r"audio_2.wav", "text": r"text_2.txt"},
    {"audio": r"audio_3.wav", "text": r"text_3.txt"},
    {"audio": r"audio_4.wav", "text": r"text_4.txt"},
    {"audio": r"audio_5.wav", "text": r"text_5.txt"},
    {"audio": r"audio_6.wav", "text": r"text_6.txt"}
]
data2 = [
    {"audio": r"audio_test_1.wav", "text": r"text_test_1.txt"}
]
data3 = [
    {"audio": r"audio_test_2.wav", "text": r"text_test_2.txt"},
]
# Load the custom dataset
def load_custom_dataset(data):
    audio_paths = []
    texts = []

    for sample in data:
        audio_path = sample["audio"]
        text = sample["text"]

        # Load audio using librosa
        audio_array, sampling_rate = librosa.load(audio_path, sr=16000)  # Resample to 16kHz
        audio_paths.append({"array": audio_array, "sampling_rate": sampling_rate})
        texts.append(text)

    return Dataset.from_dict({"audio": audio_paths, "text": texts})
def load_custom_dataset(data):
    audio_paths = []
    texts = []

    for sample in data:
        audio_path = sample["audio"]
        text_file_path = sample["text"]  # Assuming this is the path to the text file

        # Load audio using librosa
        audio_array, sampling_rate = librosa.load(audio_path, sr=16000)  # Resample to 16kHz
        audio_paths.append({"array": audio_array, "sampling_rate": sampling_rate})

        # Read the actual text from the file
        if os.path.exists(text_file_path):
            with open(text_file_path, 'r') as file:
                text_content = file.read().strip()  # Read and strip any extra whitespace
            texts.append(text_content)
        else:
            print(f"Text file not found: {text_file_path}")
            texts.append("")  # Append empty string if file not found

    return Dataset.from_dict({"audio": audio_paths, "text": texts})
train_dataset = load_custom_dataset(data1)
val_dataset = load_custom_dataset(data2)
test_dataset = load_custom_dataset(data3)"""

'data1 = [\n    {"audio": r"audio_1.wav", "text": r"text_1.txt"},\n    {"audio": r"audio_2.wav", "text": r"text_2.txt"},\n    {"audio": r"audio_3.wav", "text": r"text_3.txt"},\n    {"audio": r"audio_4.wav", "text": r"text_4.txt"},\n    {"audio": r"audio_5.wav", "text": r"text_5.txt"},\n    {"audio": r"audio_6.wav", "text": r"text_6.txt"}\n]\ndata2 = [\n    {"audio": r"audio_test_1.wav", "text": r"text_test_1.txt"}\n]\ndata3 = [\n    {"audio": r"audio_test_2.wav", "text": r"text_test_2.txt"},\n]\n# Load the custom dataset\ndef load_custom_dataset(data):\n    audio_paths = []\n    texts = []\n\n    for sample in data:\n        audio_path = sample["audio"]\n        text = sample["text"]\n\n        # Load audio using librosa\n        audio_array, sampling_rate = librosa.load(audio_path, sr=16000)  # Resample to 16kHz\n        audio_paths.append({"array": audio_array, "sampling_rate": sampling_rate})\n        texts.append(text)\n\n    return Dataset.from_dict({"audio": audio_paths, "text": 

In [ ]:
from google.colab import drive
import os
# Mount the drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Loading the dataset Using Drive**

In [ ]:
from google.colab import drive
import os
import librosa

# put path with your actual folder path in Google Drive
drive_folder_audio = "/content/drive/MyDrive/Project data/voice"
drive_folder_text = "/content/drive/MyDrive/Project data/text"
# List audio and text files
audio_files = [f for f in os.listdir(drive_folder_audio) if f.endswith('.wav')]
text_files = [f for f in os.listdir(drive_folder_text) if f.endswith('.txt')]

# Combining audio and text files into a dataset structure
data = [{"audio": os.path.join(drive_folder_audio, audio_file),
         "text": os.path.join(drive_folder_text, text_file)}
        for audio_file, text_file in zip(audio_files, text_files)]

# Load the custom dataset
def load_custom_dataset(data):
    audio_paths = []
    texts = []

    for sample in data:
        audio_path = sample["audio"]
        text_file_path = sample["text"]  # Assuming this is the path to the text file

        # Load audio using librosa
        audio_array, sampling_rate = librosa.load(audio_path, sr=16000)  # Resample to 16kHz
        audio_paths.append({"array": audio_array, "sampling_rate": sampling_rate})

        # Read the actual text from the file
        if os.path.exists(text_file_path):
            with open(text_file_path, 'r') as file:
                text_content = file.read().strip()  # Read and strip any extra whitespace
            texts.append(text_content)
        else:
            print(f"Text file not found: {text_file_path}")
            texts.append("")  # Append empty string if file not found

    return Dataset.from_dict({"audio": audio_paths, "text": texts})

dataset = load_custom_dataset(data)

In [ ]:
"""from google.colab import drive
import os
# Mount the drive
drive.mount('/content/drive')

# put path with your actual folder path in Google Drive
drive_folder = "/content/drive/MyDrive/audio_train"

# List audio and text files
audio_files = [f for f in os.listdir(drive_folder) if f.endswith('.wav')]
text_files = [f for f in os.listdir(drive_folder) if f.endswith('.txt')]

# Combining audio and text files into a dataset structure
data = [{"audio": os.path.join(drive_folder, audio_file),
         "text": os.path.join(drive_folder, text_file)}
        for audio_file, text_file in zip(audio_files, text_files)]

# Load the custom dataset
def load_custom_dataset(data):
    audio_paths = []
    texts = []

    for sample in data:
        audio_path = sample["audio"]
        text_file_path = sample["text"]  # Assuming this is the path to the text file

        # Load audio using librosa
        audio_array, sampling_rate = librosa.load(audio_path, sr=16000)  # Resample to 16kHz
        audio_paths.append({"array": audio_array, "sampling_rate": sampling_rate})

        # Read the actual text from the file
        if os.path.exists(text_file_path):
            with open(text_file_path, 'r') as file:
                text_content = file.read().strip()  # Read and strip any extra whitespace
            texts.append(text_content)
        else:
            print(f"Text file not found: {text_file_path}")
            texts.append("")  # Append empty string if file not found

    return Dataset.from_dict({"audio": audio_paths, "text": texts})

train_dataset = load_custom_dataset(data)
# Mount the drive
drive.mount('/content/drive')

# put path with your actual folder path in Google Drive
drive_folder = "/content/drive/MyDrive/audio_val"

# List audio and text files
audio_files = [f for f in os.listdir(drive_folder) if f.endswith('.wav')]
text_files = [f for f in os.listdir(drive_folder) if f.endswith('.txt')]

# Combining audio and text files into a dataset structure
data = [{"audio": os.path.join(drive_folder, audio_file),
         "text": os.path.join(drive_folder, text_file)}
        for audio_file, text_file in zip(audio_files, text_files)]

val_dataset = load_custom_dataset(data)

# Mount the drive
drive.mount('/content/drive')

# put path with your actual folder path in Google Drive
drive_folder = "/content/drive/MyDrive/audio_test"

# List audio and text files
audio_files = [f for f in os.listdir(drive_folder) if f.endswith('.wav')]
text_files = [f for f in os.listdir(drive_folder) if f.endswith('.txt')]

# Combining audio and text files into a dataset structure
data = [{"audio": os.path.join(drive_folder, audio_file),
         "text": os.path.join(drive_folder, text_file)}
        for audio_file, text_file in zip(audio_files, text_files)]

test_dataset = load_custom_dataset(data)"""

'from google.colab import drive\nimport os\n# Mount the drive\ndrive.mount(\'/content/drive\')\n\n# put path with your actual folder path in Google Drive\ndrive_folder = "/content/drive/MyDrive/audio_train"\n\n# List audio and text files\naudio_files = [f for f in os.listdir(drive_folder) if f.endswith(\'.wav\')]\ntext_files = [f for f in os.listdir(drive_folder) if f.endswith(\'.txt\')]\n\n# Combining audio and text files into a dataset structure\ndata = [{"audio": os.path.join(drive_folder, audio_file),\n         "text": os.path.join(drive_folder, text_file)}\n        for audio_file, text_file in zip(audio_files, text_files)]\n\n# Load the custom dataset\ndef load_custom_dataset(data):\n    audio_paths = []\n    texts = []\n\n    for sample in data:\n        audio_path = sample["audio"]\n        text_file_path = sample["text"]  # Assuming this is the path to the text file\n\n        # Load audio using librosa\n        audio_array, sampling_rate = librosa.load(audio_path, sr=16000)  #

**Representation of the dataset structure after loading**


In [ ]:
type(dataset)

datasets.arrow_dataset.Dataset

In [ ]:
print(dataset)

Dataset({
    features: ['audio', 'text'],
    num_rows: 13
})


In [ ]:
dataset.features

{'audio': {'array': Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None),
  'sampling_rate': Value(dtype='int64', id=None)},
 'text': Value(dtype='string', id=None)}

In [ ]:
dataset[0]

{'audio': {'array': [0.00022152811288833618,
   0.00022612139582633972,
   0.0007825568318367004,
   0.0012400895357131958,
   0.0021731331944465637,
   0.002986973151564598,
   0.004667278379201889,
   0.005975440144538879,
   0.008551908656954765,
   0.00927615538239479,
   0.012629199773073196,
   0.01453055813908577,
   0.019051602110266685,
   0.024416876956820488,
   0.025739502161741257,
   0.033518292009830475,
   0.034028615802526474,
   0.03696601837873459,
   0.041832007467746735,
   0.04626687988638878,
   0.04825130105018616,
   0.05005871132016182,
   0.05507557839155197,
   0.06676404923200607,
   0.07942528277635574,
   0.07984527200460434,
   0.08121095597743988,
   0.07959476858377457,
   0.08094972372055054,
   0.0820411965250969,
   0.0786353200674057,
   0.0788152888417244,
   0.07994366437196732,
   0.08487851172685623,
   0.08139704167842865,
   0.07914361357688904,
   0.08009975403547287,
   0.07851693034172058,
   0.08143061399459839,
   0.08070893585681915,
  

**Splitting the dataset into train and test sets**

In [ ]:
train_val_split_1 = dataset.train_test_split(test_size=1)
train_val_split_2 = train_val_split_1['train'].train_test_split(test_size=4)
train_dataset = train_val_split_2['train']
val_dataset = train_val_split_2['test']
test_dataset = train_val_split_1['test']

In [ ]:
test_dataset

Dataset({
    features: ['audio', 'text'],
    num_rows: 1
})

# Data Cleaning and Preprocessing

In [ ]:
def preprocess_function(batch):
    # To hold the expanded input features and labels
    all_input_features = []
    all_labels = []

    # Process each example in the batch
    for i in range(len(batch["audio"])):
        # Access the audio array and corresponding text for the single example
        audio = batch["audio"][i]["array"]
        text = batch["text"][i]

        # Noise reduction, trimming, and normalization
        audio = nr.reduce_noise(y=audio, sr=16000)
        audio, _ = librosa.effects.trim(audio, top_db=20)
        audio = librosa.util.normalize(audio)

        # Set maximum length for a 30-second chunk
        sample_rate = 16000
        max_length = sample_rate * 30  # 30 seconds in samples
        overlap_duration = 1  # seconds
        overlap_samples = overlap_duration * sample_rate  # Convert overlap duration to samples

        # Create overlapping audio chunks
        audio_chunks = []
        for j in range(0, len(audio), max_length - overlap_samples):
            chunk = audio[j:j + max_length]
            if len(chunk) < max_length:  # If chunk is shorter than 30 seconds, pad it
                padding = max_length - len(chunk)
                chunk = np.pad(chunk, (0, padding), mode='constant')
            audio_chunks.append(chunk)

        # Data Cleaning for the Text Data
        # Step 1: Replace multiple spaces with a single space
        text = re.sub(r'\s+', ' ', text)
        # Step 2: Strip leading and trailing spaces
        text = text.strip()
        # Step 3: Remove unwanted characters (keeping only letters, numbers, spaces, and basic punctuation)
        text = re.sub(r'[^a-zA-Z0-9.,!?\'" ]', '', text)
        # Step 4: Handle newline characters if necessary (e.g., replace with spaces)
        text = text.replace('\n', ' ')  # Replace newlines with spaces
        # Step 5: Optionally, convert to lowercase (if case-insensitivity is needed)
        text = text.lower()

        # Tokenize the entire text first
        tokenized_text = processor.tokenizer(text, padding=True, truncation=True, return_tensors="pt").input_ids.squeeze(0)
        # Estimate words per second based on the length of the audio and text
        if len(audio) > 0:
            words_per_second = len(text.split()) / (len(audio) / sample_rate)  # Calculate words per second
        else:
            words_per_second = 0

        # Calculate the number of words per 30-second chunk
        words_per_chunk = int(words_per_second * 30)
        tokens_per_chunk = words_per_chunk  # This is an approximation

        # Ensure that the overlap does not exceed tokens_per_chunk
        overlap_size = int(overlap_samples * (words_per_second / sample_rate))
        if tokens_per_chunk <= overlap_size:
            step_size = 1  # Prevent step size from being zero
        else:
            step_size = tokens_per_chunk - overlap_size

        # Create overlapping text chunks
        text_chunks = []
        for k in range(0, len(tokenized_text), step_size):
            chunk = tokenized_text[k:k + tokens_per_chunk]
            if len(chunk) > 0:  # Only append non-empty chunks
                text_chunks.append(chunk)

        # Ensure alignment between audio chunks and text chunks
        while len(audio_chunks) > len(text_chunks):
            text_chunks.append(np.zeros(tokens_per_chunk))  # Add zero-padded text chunks if necessary
        while len(audio_chunks) < len(text_chunks):
            audio_chunks.append(np.zeros(max_length))  # Add zero-padded audio chunks if necessary

        # Process each audio-text chunk pair
        for audio_chunk, text_chunk in zip(audio_chunks, text_chunks):
            inputs = processor(audio_chunk, sampling_rate=sample_rate, padding=True,truncation=True, return_tensors="pt")  # Extract audio features

            # Append each chunk's features and labels to the expanded lists
            all_input_features.append(inputs.input_features[0].numpy())
            all_labels.append(text_chunk.tolist())  # Append the already tokenized text chunk

    # Check if any features were collected
    print(f"Preprocessed {len(all_input_features)} Audio Chunk and {len(all_labels)} Text(Tokens) Chunk.")
    # Return the expanded dictionary
    return {
        "input_features": all_input_features,
        "labels": all_labels
    }

train_dataset_prep = train_dataset.map(preprocess_function, batched=True, remove_columns=train_dataset.column_names)
val_dataset_prep = val_dataset.map(preprocess_function, batched=True, remove_columns=val_dataset.column_names)
test_dataset_prep = test_dataset.map(preprocess_function, batched=True, remove_columns=test_dataset.column_names)

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

Preprocessed 375 Audio Chunk and 375 Text(Tokens) Chunk.


Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Preprocessed 147 Audio Chunk and 147 Text(Tokens) Chunk.


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Preprocessed 38 Audio Chunk and 38 Text(Tokens) Chunk.


**Representation of the preprocessed datasets structure and data inside it**

In [ ]:
test_dataset_prep

Dataset({
    features: ['input_features', 'labels'],
    num_rows: 38
})

In [ ]:
train_dataset_prep[0]

{'input_features': [[0.5165708065032959,
   0.4629414677619934,
   0.6835030913352966,
   0.8719214797019958,
   1.004203200340271,
   1.0395317077636719,
   0.9817646145820618,
   0.7387032508850098,
   0.5448017120361328,
   0.6325464248657227,
   0.7822233438491821,
   0.8688185214996338,
   0.8549042344093323,
   0.9079149961471558,
   0.8793801665306091,
   0.5409188270568848,
   0.6361424326896667,
   0.6888201236724854,
   0.746930718421936,
   0.7846731543540955,
   0.720592737197876,
   0.5663949251174927,
   0.49912387132644653,
   0.4971070885658264,
   0.4070577621459961,
   0.2969573140144348,
   0.26436561346054077,
   0.2150474190711975,
   0.09399956464767456,
   0.04586207866668701,
   0.1585436463356018,
   0.22816681861877441,
   0.24257588386535645,
   0.11475032567977905,
   0.3447993993759155,
   0.47924351692199707,
   0.49097520112991333,
   0.410489559173584,
   0.3264387249946594,
   0.38666844367980957,
   0.4290928840637207,
   0.4132431149482727,
   0.45164

In [ ]:
token_ids = test_dataset_prep['labels'][0]  # Get the first label entry

# Check if the token_ids are in the correct format
if hasattr(token_ids, 'tolist'):
    token_ids = token_ids.tolist()  # Convert to list if it's a tensor

# Decode the token IDs to text
decoded_text = processor.tokenizer.decode(token_ids, skip_special_tokens=True)
decoded_text

'translator viviane p. reviewer amanda chu a few years ago, i began to observe something in my own behavior that made me a bit uncomfortable. and that was that from the moment that i woke up to the end of the day, my life was a series of screens. i started the day with the thing that woke'

# **Prepraring for Fine Tuning**

In [ ]:
# Whisper Architecture
print(model)

WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(80, 768, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(768, 768, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 768)
      (layers): ModuleList(
        (0-11): 12 x WhisperEncoderLayer(
          (self_attn): WhisperSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
        

In [ ]:
#PEFT FINE TUNING
lora_config = LoraConfig(
    r=16,           # Low-rank adaptation dimension
    lora_alpha=32,  # Scaling factor
    lora_dropout=0.1,  # Dropout rate for LoRA layers
    target_modules=[
        "encoder.layers.11.fc1",  # Last encoder feed-forward layer
        "encoder.layers.11.fc2",  # Last encoder feed-forward layer
        "decoder.layers.11.fc1",   # Last decoder feed-forward layer
        "decoder.layers.11.fc2"    # Last decoder feed-forward layer
    ]
)
peft_model = get_peft_model(model, lora_config)

In [ ]:
from transformers import EncoderDecoderCache


training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-finetuned-amin",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    logging_steps=5,
    save_steps=100,
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    logging_dir="./logs",
    learning_rate=5e-5,
    predict_with_generate=True,
    generation_max_length=225,
    fp16=True,  # If using GPU
    push_to_hub=False,  # You can push the model to Hugging Face Hub if needed
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
print(train_dataset_prep)

Dataset({
    features: ['input_features', 'labels'],
    num_rows: 375
})


In [ ]:
from transformers import EncoderDecoderCache, DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, EarlyStoppingCallback

# Initialize the DataCollatorForSeq2Seq with padding enabled
data_collator = DataCollatorForSeq2Seq(
    tokenizer=processor.feature_extractor,
    model=peft_model, # Pass the model for dynamic padding
    padding=True, # Enable padding
)
encoder_decoder_cache = EncoderDecoderCache.from_legacy_cache(None)
trainer = Seq2SeqTrainer(
    model=peft_model,
    args=training_args,
    train_dataset=train_dataset_prep,
    eval_dataset=val_dataset_prep,
    tokenizer=processor.feature_extractor,
    data_collator=data_collator,

)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [ ]:
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,2.121900,No log
2,1.494000,No log
3,1.219800,No log
4,1.107100,No log
5,1.033400,No log
6,1.008400,No log
7,0.950000,No log
8,0.941900,No log
9,0.940900,No log
10,0.927800,No log


TrainOutput(global_step=470, training_loss=1.1745317337360788, metrics={'train_runtime': 930.2457, 'train_samples_per_second': 4.031, 'train_steps_per_second': 0.505, 'total_flos': 1.0835223552e+18, 'train_loss': 1.1745317337360788, 'epoch': 10.0})

In [ ]:
eval_results = trainer.evaluate()

print("Evaluation results:", eval_results)

Evaluation results: {'eval_runtime': 24.2938, 'eval_samples_per_second': 6.051, 'eval_steps_per_second': 0.782, 'epoch': 10.0}


In [ ]:
trainer.save_model("./whisper-finetuned-amin")
processor.save_pretrained("./whisper-finetuned-amin")


[]

#Generate Transcripts
**to predict new audio**

In [ ]:
model_f = WhisperForConditionalGeneration.from_pretrained("./whisper-finetuned-amin")
processor_f = WhisperProcessor.from_pretrained("./whisper-finetuned-amin")

In [ ]:
print(model_f.config.pad_token_id)

50257


In [ ]:
import torch
import evaluate
wer_metric = evaluate.load("wer")
PADDING_TOKEN_ID = 50257  # Padding token ID for Whisper

def generate_transcription(batch):
    # Extract input features
    input_features = torch.tensor(batch["input_features"]).unsqueeze(0)  # Add batch dimension

    # Create an attention mask
    attention_mask = (input_features != PADDING_TOKEN_ID).long()  # Create attention mask

    # Add language token for English ("en")
    forced_language_token = processor_f.tokenizer.convert_tokens_to_ids("<|en|>")  # Convert the English token to ID

    # Generate predicted IDs with the input features, attention mask, and forced language token
    predicted_ids = model_f.generate(
        input_features,
        #attention_mask=attention_mask,
        #max_length=1474,
        forced_bos_token_id=forced_language_token  # Force English language token at the beginning
    )

    # Decode the predicted token IDs back into text
    transcription = processor_f.batch_decode(predicted_ids, skip_special_tokens=True)[0]


    return transcription


# Apply the function to the test dataset
test_dataset_prep = test_dataset_prep.map(lambda batch: {"prediction": generate_transcription(batch)})

Map:   0%|          | 0/38 [00:00<?, ? examples/s]

Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [ ]:
test_dataset_prep

Dataset({
    features: ['input_features', 'labels', 'prediction'],
    num_rows: 38
})

In [ ]:
print(f"Prediction: {test_dataset_prep[0]['prediction']}")
print(f"Prediction: {test_dataset_prep[1]['prediction']}")
print(f"Prediction: {test_dataset_prep[2]['prediction']}")

Prediction:  I'm angry. and I'm angry because, I wish I knew this when I was younger. so, I'm a neuroscientist and a lecturer. and as a neuroscientist, I'm angry because, I wish I knew this when I was younger. so, I'm a neuroscientist and a lecturer. and as a neuroscientist, I'm angry because, I wish I knew this when I was younger. so, I'm a neuroscientist and a lecturer. and as a neuroscientist, I'm angry because, I wish I knew this when I was younger. so, I'm a neuroscientist and a lecturer. and as a neuroscientist, I'm angry because, I wish I knew this when I was younger. so, I'm a neuroscientist and a lecturer. and as a neuroscientist, I'm angry because, I wish I knew this when I was younger. so, I'm a neuroscientist and a lecturer. and as a neuroscientist, I'm angry because, I wish I knew this when I was younger. so, I'm a neuroscientist and a lecturer. and as a neuroscientist, I'm angry because, I wish I knew this when I was younger. so, I'm a neuroscientist and a lecturer. and a

**Overall Word Error Rate**

In [ ]:
import evaluate

# Initialize the WER metric
wer_metric = evaluate.load("wer")

def compute_wer(batch):
    """
    Compute the Word Error Rate (WER) for a batch of transcriptions.

    Args:
    batch (dict): A dictionary containing the 'reference' and 'hypothesis' keys.

    Returns:
    dict: A dictionary containing the WER result for each entry.
    """
    # Compute WER for the predictions and references
    batch['labels'] = [processor.tokenizer.decode(tokens, skip_special_tokens=True) for tokens in batch['labels']]
    wer_result = wer_metric.compute(predictions=batch['prediction'], references=batch['labels'])
    return {'wer': [wer_result] * len(batch['prediction'])}

# Assume the dataset has two columns: 'reference' (ground truth) and 'hypothesis' (predicted)
# You may need to adjust these based on your dataset's actual column names
# Map the compute_wer function over the dataset
results = test_dataset_prep.map(compute_wer, batched=True)

# Get the WER for the entire dataset
overall_wer = sum(results['wer']) / len(results)
print(f"Overall Word Error Rate: {overall_wer:.3f} %")

Map:   0%|          | 0/38 [00:00<?, ? examples/s]

Overall Word Error Rate: 6.859 %


In [ ]:
import evaluate

wer_metric = evaluate.load("wer")

def compute_wer(batch):
    """
    Compute the Word Error Rate (WER) and detailed metrics for a batch of transcriptions.

    Args:
    batch (dict): A dictionary containing the 'reference' and 'hypothesis' keys.

    Returns:
    dict: A dictionary containing the WER result and additional metrics for each entry.
    """
    # Decode the tokenized labels to text, ensuring valid IDs
    decoded_labels = [
        processor.tokenizer.decode(tokens, skip_special_tokens=True)
        for tokens in batch['labels'] if isinstance(tokens, (list, tuple))
    ]

    # Compute WER for the predictions and references
    wer_result = wer_metric.compute(predictions=batch['prediction'], references=decoded_labels)

    # Prepare the detailed metrics
    detailed_metrics = {
        'wer': [],  # List to hold WER values
        'num_words': [],  # List to hold number of words in references
        'substitutions': [],  # List to hold number of substitutions
        'deletions': [],  # List to hold number of deletions
        'insertions': [],  # List to hold number of insertions
        'predictions': [],  # List to hold decoded predictions
        'references': []  # List to hold references
    }

    for prediction, reference in zip(batch['prediction'], decoded_labels):
        # Decode prediction if it is in token IDs
        if isinstance(prediction, (list, tuple)):
            prediction_decoded = processor.tokenizer.decode(prediction, skip_special_tokens=True)
        else:
            prediction_decoded = prediction  # Handle already decoded predictions

        # Calculate detailed metrics
        num_words = len(reference.split())

        # Calculate substitutions, deletions, and insertions
        predictions_list = prediction_decoded.split()
        reference_list = reference.split()

        substitutions = sum(1 for pred, ref in zip(predictions_list, reference_list) if pred != ref)
        deletions = len(reference_list) - len(predictions_list)  # Words in reference not in prediction
        insertions = len(predictions_list) - len(reference_list)  # Extra words in prediction

        # Append the metrics to the lists
        detailed_metrics['wer'].append(wer_result)
        detailed_metrics['num_words'].append(num_words)
        detailed_metrics['substitutions'].append(substitutions)
        detailed_metrics['deletions'].append(deletions)
        detailed_metrics['insertions'].append(insertions)
        detailed_metrics['predictions'].append(prediction_decoded)
        detailed_metrics['references'].append(reference)

    return detailed_metrics

# Assume the dataset has two columns: 'reference' (ground truth) and 'hypothesis' (predicted)
# You may need to adjust these based on your dataset's actual column names
# Map the compute_wer function over the dataset
results = test_dataset_prep.map(compute_wer, batched=True)

# Extract overall WER and detailed metrics
overall_wer = sum(result['wer'] for result in results) / len(results)
print(f"Overall Word Error Rate: {overall_wer:.3f}")

# Optionally print detailed metrics for each entry
for idx, result in enumerate(results):
    print(f"Entry {idx + 1}: WER: {result['wer']:.3f}%, Num Words: {result['num_words']}, "
          f"Substitutions: {result['substitutions']}, Deletions: {result['deletions']}, "
          f"Insertions: {result['insertions']}, Prediction: {result['predictions']}, "
          f"Reference: {result['references']}")


In [ ]:
import shutil
from google.colab import files

# Specify the path to your model directory
model_dir = '/content/whisper-finetuned-amin'

# Create a zip archive of the model directory
shutil.make_archive('/content/whisper_finetuned_amin', 'zip', model_dir)

# Download the zipped file
files.download('/content/whisper_finetuned_amin.zip'